<a href="https://colab.research.google.com/github/willstauffernorris/DS-Unit-2-Kaggle-Challenge/blob/master/Best_Kaggle_Model_LS_DS_223_assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Lambda School Data Science

*Unit 2, Sprint 2, Module 3*
# Improving my model


---

### Setup

You can work locally (follow the [local setup instructions](https://lambdaschool.github.io/ds/unit2/local/)) or on Colab (run the code cell below).

In [0]:
%%capture
import sys

# If you're on Colab:
if 'google.colab' in sys.modules:
    DATA_PATH = 'https://raw.githubusercontent.com/LambdaSchool/DS-Unit-2-Kaggle-Challenge/master/data/'
    !pip install category_encoders==2.*

# If you're working locally:
else:
    DATA_PATH = '../data/'

In [0]:
import pandas as pd
from sklearn.model_selection import train_test_split

train = pd.merge(pd.read_csv(DATA_PATH+'waterpumps/train_features.csv'), 
                 pd.read_csv(DATA_PATH+'waterpumps/train_labels.csv'))
test = pd.read_csv(DATA_PATH+'waterpumps/test_features.csv')
sample_submission = pd.read_csv(DATA_PATH+'waterpumps/sample_submission.csv')

train.shape, test.shape

((59400, 41), (14358, 40))

In [0]:
#train, val = train_test_split(train, train_size = .8, test_size = .2,
                              #stratify = train['status_group'], random_state=43)

#train.shape, val.shape, test.shape

In [0]:
### These are my baselines.
train['status_group'].value_counts(normalize=True)

functional                 0.543081
non functional             0.384242
functional needs repair    0.072677
Name: status_group, dtype: float64

In [0]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split


def wrangle(X):
  X = X.copy()
  
  X['latitude'] = X['latitude'].replace(-2e-08, 0)
  
  cols_with_zeros = ['longitude', 'latitude', 'construction_year', 'gps_height', 'population']
  
  for col in cols_with_zeros:
    X[col] = X[col].replace(0, np.nan)
    X[col + '_MISSING'] = X[col].isnull()
    
  duplicates = ['quantity_group', 'payment_type']
  X = X.drop(columns=duplicates)

  unusable_variance = ['recorded_by', 'id']
  X = X.drop(columns=unusable_variance)

  X['date_recorded'] = pd.to_datetime(X['date_recorded'], infer_datetime_format=True)

  X['year_recorded'] = X['date_recorded'].dt.year
  X['month_recorded'] = X['date_recorded'].dt.month
  X['day_recorded'] = X['date_recorded'].dt.day
  X = X.drop(columns='date_recorded')

## How many years from construction to the date recorded
  X['years'] = X['year_recorded'] - X['construction_year']
  X['years_MISSING'] = X['years'].isnull()

### Doing some qcuts action
  #X['amount_tsh'] = pd.qcut(train['amount_tsh'], 100, labels=False, duplicates='drop')


  least_important_features = ['gps_height_MISSING', 'construction_year_MISSING',
                              'longitude_MISSING', 'latitude_MISSING', 
                              'years_MISSING', 'population_MISSING', 'num_private']
  X = X.drop(columns=least_important_features)

  #### Creating a time of the year feature
  time_features_to_drop = ['day_recorded', 'month_recorded', 'year_recorded']
  X['day_of_year_recorded'] = X['day_recorded']+(X['month_recorded']*30)-30
  X = X.drop(columns=time_features_to_drop)




  ### Changing null 'gps_height' values to the mean height (around 1000, just going to round it)

  #X['gps_height'] = X['gps_height'].replace(np.nan, 1000)


  ### Getting rid of the 'group' columns (this may or may not have a good effect)
#### ditch scheme management- same as management but with more missing values
  #useless_groups = ['waterpoint_type_group', 'quality_group', 'extraction_type_group', 
                    #'management_group', 'source_class', 'extraction_type_class',
                    #'scheme_management','source_type']
  #X = X.drop(columns=useless_groups)


###I'm going to keep the region_code and ditch the region, because the region_code has more variance
  region = ['region']
  X = X.drop(columns=region)


##same with name: shouldn't make much difference what it's called. Name data should already be captured by lat/long
  waypoint_name = ['wpt_name']
  X = X.drop(columns=waypoint_name)


##### What if I just ditched all the coarse location data and relied on the specific data?
  #coarse_location = ['basin', 'region_code', 'district_code', 'lga',	'ward']
  #X = X.drop(columns=coarse_location)


  return X

train = wrangle(train)
#val = wrangle(val)
test = wrangle(test)

In [0]:
train.head(10)

,amount_tsh,funder,gps_height,installer,longitude,latitude,basin,subvillage,region_code,district_code,lga,ward,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type,extraction_type_group,extraction_type_class,management,management_group,payment,water_quality,quality_group,quantity,source,source_type,source_class,waterpoint_type,waterpoint_type_group,status_group,years,day_of_year_recorded
0,6000.0,Roman,1390.0,Roman,34.938093,-9.856322,Lake Nyasa,Mnyusi B,11,5,Ludewa,Mundindi,109.0,True,VWC,Roman,False,1999.0,gravity,gravity,gravity,vwc,user-group,pay annually,soft,good,enough,spring,spring,groundwater,communal standpipe,communal standpipe,functional,12.0,74
1,0.0,Grumeti,1399.0,GRUMETI,34.698766,-2.147466,Lake Victoria,Nyamara,20,2,Serengeti,Natta,280.0,NaN,Other,NaN,True,2010.0,gravity,gravity,gravity,wug,user-group,never pay,soft,good,insufficient,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,3.0,66
2,25.0,Lottery Club,686.0,World vision,37.460664,-3.821329,Pangani,Majengo,21,4,Simanjiro,Ngorika,250.0,True,VWC,Nyumba ya mungu pipe scheme,True,2009.0,gravity,gravity,gravity,vwc,user-group,pay per bucket,soft,good,enough,dam,dam,surface,communal standpipe multiple,communal standpipe,functional,4.0,55
3,0.0,Unicef,263.0,UNICEF,38.486161,-11.155298,Ruvuma / Southern Coast,Mahakamani,90,63,Nanyumbu,Nanyumbu,58.0,True,VWC,NaN,True,1986.0,submersible,submersible,submersible,vwc,user-group,never pay,soft,good,dry,machine dbh,borehole,groundwater,communal standpipe multiple,communal standpipe,non functional,27.0,28
4,0.0,Action In A,NaN,Artisan,31.130847,-1.825359,Lake Victoria,Kyanyamisa,18,1,Karagwe,Nyakasimbi,NaN,True,NaN,NaN,True,NaN,gravity,gravity,gravity,other,other,never pay,soft,good,seasonal,rainwater harvesting,rainwater harvesting,surface,communal standpipe,communal standpipe,functional,NaN,193
5,20.0,Mkinga Distric Coun,NaN,DWE,39.172796,-4.765587,Pangani,Moa/Mwereme,4,8,Mkinga,Moa,1.0,True,VWC,Zingibali,True,2009.0,submersible,submersible,submersible,vwc,user-group,pay per bucket,salty,salty,enough,other,other,unknown,communal standpipe multiple,communal standpipe,functional,2.0,73
6,0.0,Dwsp,NaN,DWSP,33.362410,-3.766365,Internal,Ishinabulandi,17,3,Shinyanga Rural,Samuye,NaN,True,VWC,NaN,True,NaN,swn 80,swn 80,handpump,vwc,user-group,never pay,soft,good,enough,machine dbh,borehole,groundwater,hand pump,hand pump,non functional,NaN,271
7,0.0,Rwssp,NaN,DWE,32.620617,-4.226198,Lake Tanganyika,Nyawishi Center,17,3,Kahama,Chambo,NaN,True,NaN,NaN,True,NaN,nira/tanira,nira/tanira,handpump,wug,user-group,unknown,milky,milky,enough,shallow well,shallow well,groundwater,hand pump,hand pump,non functional,NaN,279
8,0.0,Wateraid,NaN,Water Aid,32.711100,-5.146712,Lake Tanganyika,Imalauduki,14,6,Tabora Urban,Itetemia,NaN,True,VWC,NaN,True,NaN,india mark ii,india mark ii,handpump,vwc,user-group,never pay,salty,salty,seasonal,machine dbh,borehole,groundwater,hand pump,hand pump,non functional,NaN,303
9,0.0,Isingiro Ho,NaN,Artisan,30.626991,-1.257051,Lake Victoria,Mkonomre,18,1,Karagwe,Kaisho,NaN,True,NaN,NaN,True,NaN,nira/tanira,nira/tanira,handpump,vwc,user-group,never pay,soft,good,enough,shallow well,shallow well,groundwater,hand pump,hand pump,functional,NaN,213


In [0]:
train.columns

In [0]:
target = 'status_group'

#train_features = train.drop(columns=[target])
#numeric_features = train_features.select_dtypes(include='number').columns.tolist()
#cardinality = train_features.select_dtypes(exclude='number').nunique()
#categorical_features = cardinality[cardinality <=50].index.tolist()
#features = numeric_features + categorical_features

In [0]:
X_train = train.drop(columns=target)
y_train = train[target]
#X_val = val.drop(columns=target)
#y_val = val[target]
X_test = test

In [0]:
%%time
import category_encoders as ce
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from scipy.stats import randint, uniform



pipeline = make_pipeline(
    ce.OrdinalEncoder(),
    SimpleImputer(strategy='median'),
    RandomForestClassifier(n_estimators = 200, min_samples_leaf=2, random_state=42, n_jobs=-1)
)


param_distributions = {
    'simpleimputer__strategy': ['mean', 'median'], 
    #'randomforestregressor__n_estimators': randint(50, 500)
    'randomforestclassifier__max_depth': [5, 10, 15, 20, None],
    'randomforestclassifier__max_features': uniform(0, 1), 
}


search = RandomizedSearchCV(
    pipeline, 
    param_distributions=param_distributions, 
    n_iter=3, 
    cv=3, 
    scoring='neg_mean_absolute_error', 
    verbose=10, 
    return_train_score=True, 
    n_jobs=-1
)

search.fit(X_train, y_train)


print('Best hyperparameters', search.best_params_)
print('Cross-validation MAE', -search.best_score_)

pipeline = search.best_estimator_
pipeline.fit(X_train, y_train)
#print('Train Accuracy', pipeline.score(X_train, y_train))
#print(f'Validation Accuracy', pipeline.score(X_val, y_val))

y_pred = pipeline.predict(X_test)

#print('Test Accuracy', pipeline.score(X_test, y_pred))

Fitting 3 folds for each of 3 candidates, totalling 9 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done   1 tasks      | elapsed:   51.8s


ValueError: ignored

In [0]:
print(X_train.shape)
#print(X_val.shape)

In [0]:
#X_combo = pd.concat([X_train, X_val], axis=0)
#print(X_combo.shape)
##X_combo.head()

#y_combo = pd.concat([y_train, y_val], axis=0)
#print(y_combo.shape)
#vertical_stack = pd.concat([survey_sub, survey_sub_last10], axis=0)

In [0]:
#pipeline.fit(X_combo, y_combo)

In [0]:
#y_pred = pipeline.predict(X_test)

In [0]:
encoder = pipeline.named_steps['ordinalencoder']
encoded = encoder.transform(X_train)
encoded.shape
#encoded.head(20)
#encoded.info()

In [0]:
%matplotlib inline
import matplotlib.pyplot as plt

rf = pipeline.named_steps['randomforestclassifier']

importances = pd.Series(rf.feature_importances_, encoded.columns)



n = 50
plt.figure(figsize=(10,n/2))
plt.title(f'Top {n} features')
importances.sort_values()[-n:].plot.barh();

In [0]:
y_pred.shape

In [0]:

sample_submission = pd.read_csv('https://raw.githubusercontent.com/willstauffernorris/DS-Unit-2-Kaggle-Challenge/master/sample_submission.csv')
submission = sample_submission.copy()
submission['status_group'] = y_pred
submission.to_csv('willsn_submission.csv', index=False)

from google.colab import files
files.download('willsn_submission.csv')
